# --- Day 17: Two Steps Forward ---

You're trying to access a secure vault protected by a 4x4 grid of small rooms connected by doors. You start in the top-left room (marked S), and you can access the vault (marked V) once you reach the bottom-right room:

```
#########
#S| | | #
#-#-#-#-#
# | | | #
#-#-#-#-#
# | | | #
#-#-#-#-#
# | | |  
####### V
```

Fixed walls are marked with #, and doors are marked with - or |.

The doors in your current room are either open or closed (and locked) based on the hexadecimal MD5 hash of a passcode (your puzzle input) followed by a sequence of uppercase characters representing the path you have taken so far (U for up, D for down, L for left, and R for right).

Only the first four characters of the hash are used; they represent, respectively, the doors up, down, left, and right from your current position. Any b, c, d, e, or f means that the corresponding door is open; any other character (any number or a) means that the corresponding door is closed and locked.

To access the vault, all you need to do is reach the bottom-right room; reaching this room opens the vault and all doors in the maze.

For example, suppose the passcode is `hijkl`. Initially, you have taken no steps, and so your path is empty: you simply find the MD5 hash of hijkl alone. The first four characters of this hash are `ced9`, which indicate that up is open (c), down is open (e), left is open (d), and right is closed and locked (9). Because you start in the top-left corner, there are no "up" or "left" doors to be open, so your only choice is down.

Next, having gone only one step (down, or D), you find the hash of hijklD. This produces f2bc, which indicates that you can go back up, left (but that's a wall), or right. Going right means hashing hijklDR to get 5745 - all doors closed and locked. However, going up instead is worthwhile: even though it returns you to the room you started in, your path would then be DU, opening a different set of doors.

After going DU (and then hashing hijklDU to get 528e), only the right door is open; after going DUR, all doors lock. (Fortunately, your actual passcode is not hijkl).

Passcodes actually used by Easter Bunny Vault Security do allow access to the vault if you know the right path. For example:

- If your passcode were ihgpwlah, the shortest path would be DDRRRD.
- With kglvqrro, the shortest path would be DDUDRLRRUDRD.
- With ulqzkmiv, the shortest would be DRURDRUDDLLDLUURRDULRLDUUDDDRR.

**Given your vault's passcode, what is the shortest path (the actual path, not just the length) to reach the vault?**

---

This looks similar to Day 13, but different enough that its not straightforward. Obviosuly a graph problem, but how to represent it?

In [122]:
import hashlib
from collections import namedtuple, deque
import numpy as np
from copy import deepcopy

Our inputs:

In [135]:
test_input = "hijkl"
puzzle_input = "veumntbg"
maze_string = """
#########
#S| | | #
#-#-#-#-#
# | | | #
#-#-#-#-#
# | | | #
#-#-#-#-#
# | | |  
####### V"""

First up, some helper variables to figure out whats what in our maze:

In [236]:
open_path = namedtuple("Can_Go", ["U", "D", "L", "R"])
is_open = set(["b", "c", "d", "e", "f"])
moves = list("UDLR")
moves_pos = ((0,-2), (0,2),(-2,0), (2,0)) # possible moves for a 4 way grid

Putting this all into a class, since there are many moving parts so I want to wrap them all up in a convenient wrapper:

In [239]:
class Maze:
    def __init__(self, passcode, maze_string=maze_string):
        self.maze = np.array([[i for i in line] for line in maze_string.strip().split("\n")])
        self.pos = np.array([loc[0] for loc in np.where(self.maze=="S")])
        self.path = ""
        self.passcode = passcode
    
    def __repr__(self):
        return f"{self.pos}|{self.path}"
    
    def move(self, d):
        i = moves.index(d)
        if self.can_go()[i]:
            self.pos += moves_pos[i]
            self.path += d
            return True
        else:
            print("Couldn't move in direction", d)
            return False
        
    def get_doors(self, verbose=False):
        """returns directions doors are open"""
        s = self.passcode + self.path
        h = hashlib.md5(s.encode()).hexdigest()
        if verbose: print(s, h[:4])
        return open_path(*[char in is_open for char in h[:4]])
    
    def get_walls(self):
        """returns directions there are walls"""
        walls = [False for _ in range(4)]
        for i, p in enumerate(moves_pos):
            p = np.add(self.pos, p)
            if min(p) >= 0 and max(p) < len(self.maze):
                if self.maze[tuple(p)] != "#":
                    walls[i] = True
        return open_path(*walls)
    
    def can_go(self):
        """returns directions we can move to in the maze"""
        "returns directions "
        pos=self.pos
        path=self.path
        return open_path(*[d&w for (d,w) in zip(self.get_doors(), self.get_walls())])
    
    def possible_mazes(self):
        """returns a list of mazes where are movable to from this state"""
        can_move_to = list()
        for i, go in enumerate(self.can_go()):
            if go:
                new_m = deepcopy(self)
                new_m.move(moves[i])
                can_move_to.append(new_m)
        return can_move_to
        
    def in_vault(self, goal=[7,7]):
        """returns True if inside vault"""
        return np.all(self.pos==goal) # our destination
             
m = Maze(passcode=test_input)
m.maze, m.get_doors(), m.get_walls(), m.can_go(), m.in_vault(), m.passcode

(array([['#', '#', '#', '#', '#', '#', '#', '#', '#'],
        ['#', 'S', '|', ' ', '|', ' ', '|', ' ', '#'],
        ['#', '-', '#', '-', '#', '-', '#', '-', '#'],
        ['#', ' ', '|', ' ', '|', ' ', '|', ' ', '#'],
        ['#', '-', '#', '-', '#', '-', '#', '-', '#'],
        ['#', ' ', '|', ' ', '|', ' ', '|', ' ', '#'],
        ['#', '-', '#', '-', '#', '-', '#', '-', '#'],
        ['#', ' ', '|', ' ', '|', ' ', '|', ' ', ' '],
        ['#', '#', '#', '#', '#', '#', '#', ' ', 'V']], dtype='<U1'),
 Can_Go(U=True, D=True, L=True, R=False),
 Can_Go(U=False, D=True, L=False, R=True),
 Can_Go(U=False, D=True, L=False, R=False),
 False,
 'hijkl')

Now to get the shortest path by using BFS:

In [244]:
all_mazes = set() # tracking all possible valid mazes

def shortest_path(passcode=test_input, goal=[7,7], verbose=False):
    """returns the shortest path to the vault"""
    maze = Maze(passcode=passcode)
    stack = deque() # the mazes to process
    stack.append(maze)

    m = stack.popleft() # the current maze being looked at
    
    if verbose: print("Starting at maze pos", m.pos, "searching for", goal)

    i = 0 # counter
    
    while not m.in_vault(): 
        if verbose: print(f"loop {i} at {m},can go: {m.possible_mazes()}")
        stack.extend(m.possible_mazes())
        all_mazes.add(m) # have now processed this path
        #print(stack, len(stack), stack[0])

        try:
            m = stack.popleft()
        except:
            print(f"Stack is empty at loop {i}")
            print(f"At position {m.pos}, Path: {m.path}")
            return all_mazes

        i+=1
        if i % 100 == 0:
            print(f"step {i}, looking at maze: {m}")
            print(all_mazes)

    print("----"*12)
    print(f"It took {len(m.path)-1} steps using path {m.path}")
    return m.path

#shortest_path(test_input, verbose=True) #DUR
assert shortest_path("ihgpwlah", verbose=True) == "DDRRRD"

Starting at maze pos [1 1] searching for [7, 7]
loop 0 at [1 1]|,can go: [[1 3]|D, [3 1]|R]
loop 1 at [1 3]|D,can go: [[1 1]|DU, [1 5]|DD, [3 3]|DR]
loop 2 at [3 1]|R,can go: []
loop 3 at [1 1]|DU,can go: []
loop 4 at [1 5]|DD,can go: [[3 5]|DDR]
loop 5 at [3 3]|DR,can go: [[5 3]|DRR]
loop 6 at [3 5]|DDR,can go: [[3 3]|DDRU, [3 7]|DDRD, [1 5]|DDRL, [5 5]|DDRR]
loop 7 at [5 3]|DRR,can go: [[5 1]|DRRU]
loop 8 at [3 3]|DDRU,can go: [[3 5]|DDRUD]
loop 9 at [3 7]|DDRD,can go: [[3 5]|DDRDU, [1 7]|DDRDL]
loop 10 at [1 5]|DDRL,can go: [[1 7]|DDRLD, [3 5]|DDRLR]
loop 11 at [5 5]|DDRR,can go: [[5 7]|DDRRD, [7 5]|DDRRR]
loop 12 at [5 1]|DRRU,can go: [[3 1]|DRRUL]
loop 13 at [3 5]|DDRUD,can go: []
loop 14 at [3 5]|DDRDU,can go: [[3 3]|DDRDUU]
loop 15 at [1 7]|DDRDL,can go: [[1 5]|DDRDLU]
loop 16 at [1 7]|DDRLD,can go: [[1 5]|DDRLDU]
loop 17 at [3 5]|DDRLR,can go: [[3 3]|DDRLRU, [3 7]|DDRLRD]
loop 18 at [5 7]|DDRRD,can go: []
loop 19 at [7 5]|DDRRR,can go: [[7 7]|DDRRRD]
loop 20 at [3 1]|DRRUL,can 

And presto, it works! on to the actual puzzle input:

In [245]:
shortest_path(puzzle_input)

------------------------------------------------
It took 9 steps using path DDRRULRDRD


'DDRRULRDRD'

`DDRRULRDRD` is the answer to the puzzle input for part 1.

# --- Part Two ---

You're curious how robust this security solution really is, and so you decide to find longer and longer paths which still provide access to the vault. You remember that paths always end the first time they reach the bottom-right room (that is, they can never pass through it, only end in it).

For example:

- If your passcode were ihgpwlah, the longest path would take 370 steps.
- With kglvqrro, the longest path would be 492 steps long.
- With ulqzkmiv, the longest path would be 830 steps long.

**What is the length of the longest path that reaches the vault?**

---

first up, I'm going to try the simple solution of just computing all the paths and seeing the longest one. That is of course hideously inefficient, but here goes:

In [246]:
all_mazes = set() # tracking all possible complete mazes

def get_all_paths(passcode=test_input, goal=[7,7], verbose=False):
    """returns the shortest path to the vault"""
    maze = Maze(passcode=passcode)
    stack = deque() # the mazes to process
    stack.append(maze)
    
    if verbose: print("Starting at maze pos", m.pos, "searching for", goal)

    i = 0 # counter
    
    while len(stack) > 0: 
        try:
            m = stack.popleft()  # current maze being looked at
        except:
            print(f"Stack is empty at loop {i}")
            print(f"At position {m.pos}, Path: {m.path}")
            return all_mazes

        if verbose: print(f"loop {i} at {m},can go: {m.possible_mazes()}")
        
        for next_maze in m.possible_mazes():
            if next_maze.in_vault():
                all_mazes.add(m) # adding all complete paths
            else:
                stack.append(next_maze) # keep going
                
        i+=1
        if i % 10000 == 0:
            print(f"step {i}, looking at a maze with path length: {len(m.path)}")
            
    print("----"*12)
    longest_path_length = max([len(m.path) for m in all_mazes]) + 1
    print(f"The longest path took: {longest_path_length} steps")
    return longest_path_length
    
% time get_all_paths("ihgpwlah") # 370 steps

step 10000, looking at a maze with path length: 156
step 20000, looking at a maze with path length: 217
step 30000, looking at a maze with path length: 266
------------------------------------------------
The longest path took: 370 steps
CPU times: user 5.32 s, sys: 67 ms, total: 5.39 s
Wall time: 5.38 s


370

In [247]:
% time get_all_paths("kglvqrro") # 492 steps

step 10000, looking at a maze with path length: 160
step 20000, looking at a maze with path length: 211
step 30000, looking at a maze with path length: 261
step 40000, looking at a maze with path length: 334
step 50000, looking at a maze with path length: 444
------------------------------------------------
The longest path took: 492 steps
CPU times: user 8.85 s, sys: 180 ms, total: 9.03 s
Wall time: 8.99 s


492

In [248]:
%time get_all_paths(puzzle_input)

step 10000, looking at a maze with path length: 147
step 20000, looking at a maze with path length: 214
step 30000, looking at a maze with path length: 372
------------------------------------------------
The longest path took: 536 steps
CPU times: user 5.66 s, sys: 55 ms, total: 5.72 s
Wall time: 5.75 s


536

the part 2 answer is `536`. It only took 6 seconds to compute, though with a bigger graph this could take a while, so look at ways to optimize.

# Notes:

- i have a over elaborate class, could have done this much simpler and faster using a named tuple to store position and a path, and having functions which took in a maze, position and path.
- basically, seperate data and functions which transform that data. Sometimes a class is a great idea, but not sure it was over here.
- anyways, this was a good learning thingamajig.